In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 10000)
from config import sub_days

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

from fbprophet import Prophet

from tqdm.notebook import tqdm

In [27]:
sp_df = pd.read_csv('sell_prices.csv')
cal_df = pd.read_csv('calendar.csv')
train_df = pd.read_csv('sales_train_validation.csv')
sub_df = pd.read_csv('sample_submission.csv')

agg_preds = pd.read_csv('agg_preds.csv')
foods_item_preds = pd.read_csv('foods_item_preds.csv')
hobbie_item_preds = pd.read_csv('hobbie_item_preds.csv')
household_item_preds = pd.read_csv('household_item_preds.csv')

In [28]:
id_cols = ['id', 'state_id', 'store_id', 'dept_id', 'item_id']
day_cols = [col for col in train_df.columns if col.startswith('d_')]
f_cols = ['F' + str(d) for d in range(1, 29)]
cal_df['y_m'] = pd.to_datetime(cal_df.date.str[:-3] + '-01')

train_df['total_sold'] = train_df[day_cols].sum(axis=1)
train_df['state_cat_id'] = train_df['state_id'] + '_' + train_df['cat_id']
train_df['state_dept_id'] = train_df['state_id'] + '_' + train_df['dept_id']
train_df['state_item_id'] = train_df['state_id'] + '_' + train_df['item_id']
train_df['store_cat_id'] = train_df['store_id'] + '_' + train_df['cat_id']
train_df['store_dept_id'] = train_df['store_id'] + '_' + train_df['dept_id']

In [33]:
def get_perc_of_items_sold_per_dept(df):
    item_df = df.groupby(['item_id',], as_index=False)['total_sold'].sum()
    item_df.rename(columns={'total_sold':'total_item_sold'}, inplace=True)
    
    train_perc_df = df[['id', 'item_id', 'total_sold']].merge(item_df, on=['item_id'])
    train_perc_df['perc'] = (train_perc_df['total_sold'] / train_perc_df['total_item_sold']).fillna(0)
    
    return train_perc_df

def get_item_dept_preds(perc_df, item_preds_df, category):
    str_len = len(category) + 6
    item_preds_df['item_id'] = item_preds_df.id.str[:str_len]
    item_dept_preds = item_preds_df.merge(perc_df[['id','item_id','perc']], on = 'item_id')
    
    item_preds_df.drop('item_id', axis = 1, inplace=True)    
        
    item_dept_preds['id'] = item_dept_preds['id_x'].str[:str_len] + \
        item_dept_preds['id_y'].str[str_len:str_len+5] + \
        item_dept_preds['id_x'].str[str_len + 2:]
        
    mult_item_dept_preds = pd.concat([item_dept_preds['id'], 
                                      item_dept_preds[f_cols].multiply(item_dept_preds['perc'], axis = 'index')],
                                     axis=1)
    return mult_item_dept_preds

In [30]:
quants = [(0.005, 0.995), (0.025, 0.975), (0.165, 0.835), (0.250, 0.750)]

def groupby_id(df, id_col):
    grouped_df = df.groupby([id_col], as_index=False)[day_cols].sum()
    return grouped_df

def melt_and_merge(grouped_df, id_col):
    melt_df = pd.melt(grouped_df,
                      id_vars= id_col,
                      var_name = 'd',
                      value_name= 'sold')
    
    merged_df = melt_df.merge(cal_df[['d','date','wm_yr_wk']], on = 'd') \
        .sort_values([id_col, 'date']) \
        .reset_index(drop=True) \
        .rename(columns={'date':'ds', 'sold':'y'})
    
    merged_df['cap'] = 25000
    merged_df['floor'] = 0
    
    return merged_df

def prophet(merged_df, id_col, q):
    pred_lst = []
    unique_ids = merged_df[id_col].unique()
    
    if id_col == 'item_id':
        merged_df = merged_df.loc[merged_df.ds > '2014-12-31']
    
    for idx in unique_ids:
        print('Predicting {} at uncertainty level {}'.format(idx, q))
        
        subset_df = merged_df.loc[merged_df[id_col] == idx]
        
        m = Prophet(daily_seasonality=True, interval_width=q[1], yearly_seasonality=True, growth='logistic')
        m.fit(subset_df[['ds','y','cap','floor']])
        future = m.make_future_dataframe(periods=28)
        future['cap'] = 25000
        future['floor'] = 0
        
        forecast = m.predict(future)
        
        forecast = forecast.loc[forecast.ds > '2016-04-24']
        
        forecast['id'] = idx
        forecast['quantile'] = str(q[0]) + '_' + str(q[1])
        forecast.rename(columns={'yhat':'yhat_mid'}, inplace=True)
        
        pred_lst.append(forecast)
        
    return pd.concat(pred_lst).reset_index(drop=True)

def pivot_preds(pred_df):
    pivot_df = pred_df \
        .pivot(index='id', columns ='ds') \
        .reset_index()
    
    pivot_df.columns = [['id'] + list(sub_days.keys())]
    
    return pivot_df

def transform_and_predict(df, id_col):
    print('Predicting {} values'.format(id_col))
    grouped_df = groupby_id(df, id_col)
    
    merged_df = melt_and_merge(grouped_df, id_col)
    
    pred_quant_lst = []
    for q in tqdm(quants):
        pred_df = prophet(merged_df, id_col, q)
        
        pred_quant_lst.append(pred_df)
    
    pred_quant_df = pd.concat(pred_quant_lst)
    
    if id_col in ['state_id', 'store_id', 'cat_id', 'dept_id', 'item_id']:
        pred_quant_df['lower_id'] = pred_quant_df['id'] + '_X_' + pred_quant_df['quantile'].str.split('_').str[0] + '_validation'
        pred_quant_df['mid_id'] = pred_quant_df['id'] + '_X_0.500_validation'
        pred_quant_df['upper_id'] = pred_quant_df['id'] + '_X_' + pred_quant_df['quantile'].str.split('_').str[1] + '_validation'
    else:
        pred_quant_df['lower_id'] = pred_quant_df['id'] + '_' + pred_quant_df['quantile'].str.split('_').str[0] + '_validation'
        pred_quant_df['mid_id'] = pred_quant_df['id'] + '_0.500_validation'
        pred_quant_df['upper_id'] = pred_quant_df['id'] + '_' + pred_quant_df['quantile'].str.split('_').str[1] + '_validation' 

    uncert_lst = []
    levels = ['lower','mid','upper',]
    
    for lev in levels:
        yhat_col = 'yhat_' + lev
        id_col = lev + '_id'

        pred_quant_pivot = pred_quant_df[['ds', yhat_col, id_col,]] \
            .drop_duplicates() \
            .pivot(index=id_col, columns='ds').reset_index()
        
        pred_quant_pivot.columns = ['id'] + list(sub_days.keys())
        
        uncert_lst.append(pred_quant_pivot)
    #pred_pivot_df = pivot_preds(pred_quant_df)
    
    return pd.concat(uncert_lst).reset_index(drop=True)

In [34]:
train_perc_df = get_perc_of_items_sold_per_dept(train_df)

foods_item_dept_preds = get_item_dept_preds(train_perc_df, foods_item_preds, 'FOODS')
hobbie_item_dept_preds = get_item_dept_preds(train_perc_df, hobbie_item_preds, 'HOBBIES')
household_item_dept_preds = get_item_dept_preds(train_perc_df, household_item_preds, 'HOUSEHOLD')

In [35]:
foods_item_preds.columns

Index(['id', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28'],
      dtype='object')

In [36]:
val_sub_df = sub_df.loc[sub_df.id.str.contains('validation')]

In [37]:
# val_sub_df[['id']].merge(final_df[['id', 'F1']], how = 'left').sort_values('F1').to_csv('test.csv')

In [46]:
def get_state_item_aggs(df, category):
    str_len = len(category) + 6
    df['new_id'] = df.id.str[str_len+1:str_len+4] + df.id.str[:str_len] + df.id.str[str_len+5:]
    
    grouped_df = df.groupby(['new_id'], as_index=False)[f_cols].sum()
    grouped_df.rename(columns={'new_id':'id'}, inplace=True)
    
    df.drop('new_id', axis=1, inplace=True)
    
    return grouped_df

In [47]:
foods_state_item_preds = get_state_item_aggs(foods_item_dept_preds, 'FOODS')
hobbie_state_item_preds = get_state_item_aggs(hobbie_item_dept_preds, 'HOBBIES')
household_state_item_preds = get_state_item_aggs(household_item_dept_preds, 'HOUSEHOLD')

In [15]:
state_preds = transform_and_predict(train_df, 'state_id')
# store_preds = transform_and_predict(train_df, 'store_id')
# cat_preds = transform_and_predict(train_df, 'cat_id')
# dept_preds = transform_and_predict(train_df, 'dept_id')
# state_cat_preds = transform_and_predict(train_df, 'state_cat_id')
# state_dept_preds = transform_and_predict(train_df, 'state_dept_id')
# store_cat_preds = transform_and_predict(train_df, 'store_cat_id')
# store_dept_preds = transform_and_predict(train_df, 'store_dept_id')

Predicting state_id values


Predicting CA at uncertainty level (0.005, 0.995)
Predicting TX at uncertainty level (0.005, 0.995)
Predicting WI at uncertainty level (0.005, 0.995)
Predicting CA at uncertainty level (0.025, 0.975)
Predicting TX at uncertainty level (0.025, 0.975)
Predicting WI at uncertainty level (0.025, 0.975)
Predicting CA at uncertainty level (0.165, 0.835)
Predicting TX at uncertainty level (0.165, 0.835)
Predicting WI at uncertainty level (0.165, 0.835)
Predicting CA at uncertainty level (0.25, 0.75)
Predicting TX at uncertainty level (0.25, 0.75)
Predicting WI at uncertainty level (0.25, 0.75)



In [40]:
state_preds['total_id'] = 'Total_X_' + state_preds.id.str[5:]
total_preds = state_preds.groupby('total_id', as_index=False)[f_cols].sum()
total_preds.rename(columns={'total_id':'id'}, inplace=True)

In [76]:
final_val_df = pd.concat([total_preds, agg_preds, foods_item_preds, hobbie_item_preds, household_item_preds,
                      foods_item_dept_preds, hobbie_item_dept_preds, household_item_dept_preds,
                      foods_state_item_preds, hobbie_state_item_preds, household_state_item_preds
                     ]).reset_index(drop=True)

In [77]:
final_val_df.shape

(385560, 29)

In [78]:
final_eval_df = final_val_df.copy()
final_eval_df['id'] = final_eval_df['id'].str.replace('validation', 'evaluation')

In [87]:
final_df = pd.concat([final_val_df, final_eval_df])
final_df['id'] = final_df.id.str.replace('0.25', '0.250')
final_df['id'] = final_df.id.str.replace('0.75', '0.750')

In [88]:
final_df.head(100)

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,Total_X_0.005_validation,28244.976833,25210.367980,24007.852410,24509.625622,29166.002912,36491.946626,35815.892896,27410.006466,24615.903495,24438.044711,24110.248064,28674.704431,36310.588990,35509.405010,27612.263515,24615.851379,23410.180786,24450.467130,29113.386626,36005.758510,35266.069651,26828.045213,25039.682254,23913.965023,24572.268252,28993.967620,37067.667271,36238.820671
1,Total_X_0.025_validation,29327.488034,27063.392407,26539.651221,26393.743521,31182.671538,37921.732390,37511.758014,29607.935786,26994.647181,26017.619523,26418.470052,30576.950408,38009.712547,37313.715948,29414.970668,27017.957551,26352.901802,26451.064038,30995.876319,38197.701843,37398.884870,28743.735773,26524.392514,26703.005397,26943.874913,31219.104133,38412.964378,38442.557361
2,Total_X_0.165_validation,33411.385127,30445.599114,30205.170065,30206.220511,34279.660670,41486.191587,40891.578242,32739.237033,30232.925182,29775.316272,29992.327737,33863.679149,41279.882309,40791.929935,32737.181617,30129.591116,29662.752367,30029.951506,34001.147189,41252.079596,40827.749577,32531.656699,30273.887895,29942.256057,30047.229558,34293.430143,41613.669886,41384.087712
3,Total_X_0.250_validation,34056.832723,31439.781490,30916.971287,31221.328710,35155.544076,42304.108811,42072.332913,33716.404921,30951.143962,30735.035446,30805.920150,34916.845865,42380.100468,41783.385283,33297.527960,31117.734045,30619.301299,30821.368579,34867.918911,42250.293959,41859.389537,33792.868355,31007.737528,30803.415635,31201.614662,35303.044055,42786.474582,42440.127375
4,Total_X_0.500_validation,38574.170886,36027.076842,35607.752669,35744.582978,39709.851013,47018.614188,46550.970281,38246.908567,35717.698049,35317.306974,35474.413084,39461.639076,46794.409715,46353.198032,38078.355583,35581.479731,35216.812859,35413.233752,39443.476288,46822.972558,46432.088352,38210.946478,35770.791160,35465.388637,35723.024231,39815.730952,47258.146511,46929.766323
5,Total_X_0.750_validation,43032.937036,40626.886285,40160.947398,40457.965542,44292.889235,51519.445011,51151.176157,42940.055890,40374.146369,40174.244033,40053.661951,44035.048700,51408.457243,51079.988183,42611.715643,40030.417924,39775.373645,39889.282373,43789.796926,51486.249279,51263.962817,42826.206226,40580.954659,40091.743970,40401.311268,44370.823505,51796.147578,51692.280107
6,Total_X_0.835_validation,44034.166968,41508.513578,41211.024524,41461.761311,45096.032405,52541.790418,51920.738831,43758.160948,41307.645213,40815.302966,41058.069413,44971.565082,52410.371463,51756.535364,43397.591117,41168.016341,40466.981381,41003.791823,44959.969623,52354.549669,51861.380805,43521.838255,41180.691903,40906.292397,41183.502031,45158.815682,52691.852434,52550.508748
7,Total_X_0.975_validation,47770.386141,44693.974333,45211.472387,44770.304902,48562.378901,55885.084169,55428.945732,47212.823490,44656.917135,44352.220085,44343.540983,48259.228444,55259.273952,55143.849689,47099.978568,44703.051955,44009.019612,44814.612008,48641.768403,55337.155062,55418.930413,47290.805394,44470.431478,44212.403219,44307.127232,49200.611881,56492.664178,56212.862864
8,Total_X_0.995_validation,49827.308743,47046.157700,45948.869674,46831.160642,50317.304609,58371.348642,57488.750888,49564.720327,46367.512945,46787.967721,46157.536927,51092.797065,58101.776699,56950.970362,48176.126406,46479.596577,45557.714315,46035.752659,50663.120164,58027.771672,57143.957427,49589.437978,45978.405694,46304.156075,46627.573463,50717.948149,57981.556092,57887.382624
9,CA_X_0.005_validation,12680.404550,11725.170800,11098.367850,10976.363090,13326.256040,16365.131560,16415.300930,12750.124560,11568.385530,11160.259980,11303.894560,12803.764120,16184.497240,16840.143760,12443.579980,11459.945860,10826.866690,10869.391250,12379.794190,16456.925230,16691.419310,12603.460650,11291.610670,11006.746430,11436.196870,12559.671810,15960.638520

In [89]:
final_df.to_csv('prophet_sub1.csv', index=False)

In [ ]:
# agg_preds = pd.concat([state_preds, store_preds, cat_preds, dept_preds, state_cat_preds, 
#            state_dept_preds, store_cat_preds, store_dept_preds]).reset_index(drop=True)

# agg_preds.iloc[:, 1:] = agg_preds.iloc[:, 1:].clip(lower=0)

# agg_preds.to_csv('agg_preds.csv', index=False)

In [ ]:
# hobbie_item_preds = transform_and_predict(train_df.loc[train_df.cat_id == 'HOBBIES'], 'item_id')
# foods_item_preds = transform_and_predict(train_df.loc[train_df.cat_id == 'FOODS'], 'item_id')
# household_item_preds = transform_and_predict(train_df.loc[train_df.cat_id == 'HOUSEHOLD'], 'item_id')

In [ ]:
item_preds = pd.read_csv('item_preds.csv')
item_preds.rename(columns={'id':'item_id'}, inplace=True)

In [ ]:
prod_df = train_perc_df[['id', 'item_id', 'perc']].merge(item_preds, how = 'left', on='item_id')

In [ ]:
prod_df.head()

In [ ]:
#prod_df.update(prod_df.iloc[:, 3:5].mul(prod_df['perc']))

In [ ]:
final_prod_df = pd.concat([prod_df['id'], prod_df[f_cols].multiply(prod_df['perc'], axis = 'index')], axis=1)

In [ ]:
final_prod_df[f_cols]

In [ ]:
np.quantile(final_prod_df[f_cols], 0.005, axis = 1)

In [ ]:
# state_graph_df = merged_train.groupby(['y_m', 'state_id'], as_index=False)['sold'].sum()
# dept_graph_df = merged_train.groupby(['y_m', 'dept_id'], as_index=False)['sold'].sum()

# sns.lineplot(x='y_m', y='sold', hue='state_id', data=state_graph_df[state_graph_df.y_m != '2011-01-01'])

# sns.lineplot(x='y_m', y='sold', hue='dept_id', data=dept_graph_df[dept_graph_df.y_m != '2011-01-01'])

In [ ]:
merged_train.columns = ['id', 'state_id', 'store_id', 'dept_id', 'item_id', 'd', 'y', 'ds', 'wm_yr_wk',
                        'weekday', 'wday', 'month', 'year', 'event_name_1', 'event_type_1', 'event_name_2',
                        'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'y_m']

In [ ]:
#m = Prophet(daily_seasonality=True, interval_width=0.8)
#m.add_regressor('id')

#m.fit(merged_train[['ds','y',]])
#future = m.make_future_dataframe(periods=365)
#forecast = m.predict(future)
#forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
#fig1 = m.plot(forecast)

In [ ]:
sp_df.shape

In [ ]:
merged_train.shape

In [ ]:
mt_df = merged_train.merge(sp_df, how = 'left', on = ['wm_yr_wk', 'item_id', 'store_id'])

In [ ]:
state_dummies = pd.get_dummies(merged_train['state_id'], drop_first=True)
dept_dummies = pd.get_dummies(merged_train['dept_id'], drop_first=True)

In [ ]:
tr_df = pd.concat([mt_df.loc[:,['id','d','y','wday','month','snap_CA','snap_TX','snap_WI','sell_price',]],
                   state_dummies, dept_dummies], axis=1)

In [ ]:
mean_sp_df = tr_df.groupby(['id'], as_index = False)['sell_price'] \
    .mean() \
    .rename(columns={'sell_price':'mean_sell_price'})

In [ ]:
tr_df = tr_df.merge(mean_sp_df, on = 'id')

In [ ]:
tr_df['sell_price'] = np.where(pd.isnull(tr_df['sell_price']), tr_df['mean_sell_price'], tr_df['sell_price'])

In [ ]:
tr_df.info(verbose=True, null_counts=True)

In [ ]:
tr_df.drop('mean_sell_price', axis = 1, inplace=True)

In [ ]:
tr_df.head()

In [ ]:
#rf = RandomForestRegressor(n_estimators=10)
import xgboost as xgb
xgb_model = xgb.XGBRegressor(objective ='reg:linear')

X = tr_df.iloc[:,3:]
y = tr_df.y

In [ ]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [ ]:
xgb_model.fit(X, y)

preds = xgb_model.predict(X_test)

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42)
#rf.fit(X, y)

In [ ]:
# from sklearn.ensemble import GradientBoostingRegressor

# model_0005 = GradientBoostingRegressor(loss="quantile", alpha=0.0005, n_estimators=5)

In [ ]:
# model_0005.fit(X, y)

In [ ]:
sub_item_val_df = sub_df.loc[(sub_df.id.str.split('_').apply(len) == 7) &
                             (sub_df.id.str.contains('validation'))
                            ].sort_values('id')

In [ ]:
melt_sub_item_val_df = pd.melt(sub_item_val_df, id_vars=['id'], var_name='vd', value_name='sold')
melt_sub_item_val_df['sub_days'] = melt_sub_item_val_df.vd.apply(lambda x: sub_days[x][0])
melt_sub_item_val_df.shape

In [ ]:
melt_sub_item_val_df['join_id'] = melt_sub_item_val_df.id.str[:-16] + 'validation'
merged_melt_sub_item_val_df = melt_sub_item_val_df.merge(prod_df, left_on='join_id', right_on='id')
merged_melt_sub_item_val_df = merged_melt_sub_item_val_df.merge(cal_df, left_on = 'sub_days', right_on = 'd')

In [ ]:
merged_melt_sub_item_val_df.head()

In [ ]:
final_val_df = merged_melt_sub_item_val_df[['id_x', 'vd', 'sold', 'wday', 'month', 'snap_CA', 'snap_TX', 'snap_WI',
                                            'state_id', 'dept_id','store_id', 'cat_id',]]

In [ ]:
final_val_df.rename(columns={'id_x':'id'}, inplace=True)

In [ ]:
final_val_df = pd.concat([final_val_df,
           pd.get_dummies(final_val_df.state_id, drop_first = True),
           pd.get_dummies(final_val_df.dept_id, drop_first = True)],
          axis = 1
         )#.drop(['dept_id','state_id'], axis = 1)

In [ ]:
prod_df.head()

In [ ]:
X_test = final_val_df.iloc[:,3:].drop(['state_id', 'dept_id', 'store_id', 'cat_id'], axis = 1)
preds = rf.predict(X_test)
final_val_df['sold'] = preds

In [ ]:
val_sub_df = final_val_df[['id','vd','sold',]].pivot(index='id', columns ='vd')['sold'].reset_index()[['id'] + list(sub_days.keys())]

In [ ]:
val_sub_df['state_id'] = val_sub_df.id.str.split('_').str[3]
val_sub_df['store_id'] = val_sub_df.id.str.split('_').str[3] + '_' + val_sub_df.id.str.split('_').str[4]
val_sub_df['cat_id'] = val_sub_df.id.str.split('_').str[0]
val_sub_df['dept_id'] = val_sub_df.id.str.split('_').str[0] + '_' + val_sub_df.id.str.split('_').str[1]

In [ ]:
eval_sub_df = val_sub_df.copy()

In [ ]:
eval_sub_df['id'] = eval_sub_df.id.str.replace('validation', 'evaluation')

In [ ]:
eval_sub_df.head()

In [ ]:
merged_sub_df = pd.concat([val_sub_df, eval_sub_df])
merged_sub_df['uncertainty'] = merged_sub_df['id'].str.split('_').str[5] + '_' + merged_sub_df['id'].str.split('_').str[6]

In [ ]:
total_agg = merged_sub_df.groupby(['uncertainty'], as_index=False).sum()
total_agg['id'] = 'Total_X_' + total_agg['uncertainty']
total_agg = total_agg[['id'] + [col for col in dept_agg.columns if col.startswith('F')]]

In [ ]:
state_agg = merged_sub_df.groupby(['state_id', 'uncertainty'], as_index=False).sum()
state_agg['id'] = state_agg['state_id'] + '_X_' + state_agg['uncertainty']
state_agg = state_agg[['id'] + [col for col in dept_agg.columns if col.startswith('F')]]

In [ ]:
store_agg = merged_sub_df.groupby(['store_id', 'uncertainty'], as_index=False).sum()
store_agg['id'] = store_agg['store_id'] + '_X_' + store_agg['uncertainty']
store_agg = store_agg[['id'] + [col for col in store_agg.columns if col.startswith('F')]]

In [ ]:
cat_agg = merged_sub_df.groupby(['cat_id', 'uncertainty'], as_index=False).sum()
cat_agg['id'] = cat_agg['cat_id'] + '_X_' + cat_agg['uncertainty']
cat_agg = cat_agg[['id'] + [col for col in cat_agg.columns if col.startswith('F')]]

In [ ]:
dept_agg = merged_sub_df.groupby(['dept_id', 'uncertainty'], as_index=False).sum()
dept_agg['id'] = dept_agg['dept_id'] + '_X_' + dept_agg['uncertainty']
dept_agg = dept_agg[['id'] + [col for col in dept_agg.columns if col.startswith('F')]]

In [ ]:
merged_sub_df['state_cat'] = merged_sub_df['state_id'] + '_' + merged_sub_df['cat_id']
state_cat_agg = merged_sub_df.groupby(['state_cat', 'uncertainty'], as_index=False).sum()
state_cat_agg['id'] = state_cat_agg['state_cat'] + '_' + state_cat_agg['uncertainty']
state_cat_agg = state_cat_agg[['id'] + [col for col in state_cat_agg.columns if col.startswith('F')]]

In [ ]:
merged_sub_df['state_dept'] = merged_sub_df['state_id'] + '_' + merged_sub_df['dept_id']
state_dept_agg = merged_sub_df.groupby(['state_dept', 'uncertainty'], as_index=False).sum()
state_dept_agg['id'] = state_dept_agg['state_dept'] + '_' + state_dept_agg['uncertainty']
state_dept_agg = state_dept_agg[['id'] + [col for col in state_dept_agg.columns if col.startswith('F')]]

In [ ]:
merged_sub_df['store_cat'] = merged_sub_df['store_id'] + '_' + merged_sub_df['cat_id']
store_cat_agg = merged_sub_df.groupby(['store_cat', 'uncertainty'], as_index=False).sum()
store_cat_agg['id'] = store_cat_agg['store_cat'] + '_' + store_cat_agg['uncertainty']
store_cat_agg = store_cat_agg[['id'] + [col for col in store_cat_agg.columns if col.startswith('F')]]

In [ ]:
merged_sub_df['store_dept'] = merged_sub_df['store_id'] + '_' + merged_sub_df['dept_id']
store_dept_agg = merged_sub_df.groupby(['store_dept', 'uncertainty'], as_index=False).sum()
store_dept_agg['id'] = store_dept_agg['store_dept'] + '_' + store_dept_agg['uncertainty']
store_dept_agg = store_dept_agg[['id'] + [col for col in store_dept_agg.columns if col.startswith('F')]]

In [ ]:
merged_sub_df['item'] = merged_sub_df.id.str.split('_').str[0] + '_' \
    + merged_sub_df.id.str.split('_').str[1] + '_' + merged_sub_df.id.str.split('_').str[2]

merged_sub_df['state_item'] = merged_sub_df['state_id'] + '_' + merged_sub_df['item']

state_item_agg = merged_sub_df.groupby(['state_item', 'uncertainty'], as_index=False).sum()
state_item_agg['id'] = state_item_agg['state_item'] + '_' + state_item_agg['uncertainty']
state_item_agg = state_item_agg[['id'] + [col for col in state_item_agg.columns if col.startswith('F')]]

In [ ]:
#Total
#State
#Store
#Category
#Dept
#Item

In [ ]:
final_sub_df = pd.concat([total_agg, state_agg, store_agg, cat_agg, dept_agg, 
                          state_cat_agg, state_dept_agg, store_cat_agg, store_dept_agg,
                          item_agg, state_item_agg,
                          merged_sub_df])

In [ ]:
final_sub_df.shape

In [ ]:
final_sub_df = final_sub_df[['id'] + [col for col in state_item_agg.columns if col.startswith('F')]].reset_index(drop=True)

In [ ]:
final_sub_df.head()

In [ ]:
final_sub_df.to_csv('sub_v3.csv', index=False)